# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://7339ef3411c8:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/25 13:19:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from regalado_floriano.spark_utils import SparkUtils

In [12]:
from pyspark.sql.functions import col, from_json, get_json_object

In [44]:
# Build schema
# Import your module
#prob lo haria diferente en el futuro
schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])


df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")



In [57]:
df_agency_names = df_agencies.select(df_agencies.agency_id, get_json_object( df_agencies.agency_info, "$.agency_name").alias("agency_name") )
df_car_names = df_cars.select(df_cars.car_id, get_json_object( df_cars.car_info, "$.car_name").alias("car_name") )
df_customer_names = df_customers.select(df_customers.customer_id, get_json_object(df_customers.customer_info, "$.customer_name").alias("customer_name"))
df_rental_keys = df_rentals.select(df_rentals.rental_id.alias("rental_id"), 
                                   get_json_object(df_rentals.rental_info, "$.car_id").alias("car_id"),  
                                    get_json_object(df_rentals.rental_info, "$.customer_id").alias("customer_id"),
                                   get_json_object(df_rentals.rental_info, "$.agency_id").alias("agency_id")
                                  )



In [62]:
r1 = df_rental_keys.join( df_car_names, df_rental_keys["car_id"] == df_car_names["car_id"] , "left")
r1 = r1.select( r1.rental_id, r1.customer_id, r1.agency_id, r1.car_name)
r2 = r1.join(df_customer_names, r1["customer_id"] == df_customer_names["customer_id"], "left")
r2 = r2.select( r2.rental_id, r2.customer_name, r2.agency_id, r2.car_name)
r3 = r2.join(df_agency_names, r2["agency_id"] == df_agency_names["agency_id"], "left")
r3 = r3.select( r3.rental_id, r3.car_name, r3.agency_name, r3.customer_name )

r3.show()

+---------+--------------------+-------------+---------------+
|rental_id|            car_name|  agency_name|  customer_name|
+---------+--------------------+-------------+---------------+
|    11891|Wallace-Carlson M...|  NYC Rentals| Margaret Jones|
|    11892|Grimes-Green Model 8|LA Car Rental|Albert Williams|
|    11893|Stewart-Allen Mod...|      SF Cars|  Caleb Fleming|
|    11894|  Campos PLC Model 4|  NYC Rentals|  Andrew Butler|
|    11895|  Wagner LLC Model 1|      SF Cars|  Kristin Potts|
|    11896|Jones, Jefferson ...|LA Car Rental|   Jeremy Parks|
|    11897|Lopez and Sons Mo...| Zapopan Auto|    Terry Wells|
|    11898| Salazar Ltd Model 8|      SF Cars|  Marc Williams|
|    11899|Villanueva PLC Mo...|LA Car Rental| Danny Williams|
|    11900|Faulkner-Howard M...|      SF Cars| Eric Owens PhD|
|    11901|Faulkner-Howard M...|  NYC Rentals|    Laura Perry|
|    11902|Faulkner-Howard M...|  NYC Rentals|     Paul Brown|
|    11903|Atkinson Ltd Mode...| Zapopan Auto|Alexa Her

25/09/25 14:59:40 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.3: worker lost: Not receiving heartbeat for 60 seconds


In [ ]:
 sc.stop()